# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f6a8c4d2040>
├── 'a' --> tensor([[ 0.7525, -0.1259,  0.0254],
│                   [-0.0361,  0.7473,  1.2433]])
└── 'x' --> <FastTreeValue 0x7f6a8c4d2700>
    └── 'c' --> tensor([[ 0.7399, -0.1499, -1.7815,  0.3320],
                        [-1.0493, -0.0887,  0.1404,  1.7528],
                        [-0.2755,  0.8267,  0.8030, -0.5056]])

In [4]:
t.a

tensor([[ 0.7525, -0.1259,  0.0254],
        [-0.0361,  0.7473,  1.2433]])

In [5]:
%timeit t.a

71.3 ns ± 0.879 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f6a8c4d2040>
├── 'a' --> tensor([[ 0.7583,  0.1554, -0.5816],
│                   [ 0.5522,  0.1475,  1.2029]])
└── 'x' --> <FastTreeValue 0x7f6a8c4d2700>
    └── 'c' --> tensor([[ 0.7399, -0.1499, -1.7815,  0.3320],
                        [-1.0493, -0.0887,  0.1404,  1.7528],
                        [-0.2755,  0.8267,  0.8030, -0.5056]])

In [7]:
%timeit t.a = new_value

74.4 ns ± 0.883 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.7525, -0.1259,  0.0254],
               [-0.0361,  0.7473,  1.2433]]),
    x: Batch(
           c: tensor([[ 0.7399, -0.1499, -1.7815,  0.3320],
                      [-1.0493, -0.0887,  0.1404,  1.7528],
                      [-0.2755,  0.8267,  0.8030, -0.5056]]),
       ),
)

In [10]:
b.a

tensor([[ 0.7525, -0.1259,  0.0254],
        [-0.0361,  0.7473,  1.2433]])

In [11]:
%timeit b.a

66.6 ns ± 1.32 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.4817, -1.9688,  0.5573],
               [-0.5830,  0.9627, -0.8452]]),
    x: Batch(
           c: tensor([[ 0.7399, -0.1499, -1.7815,  0.3320],
                      [-1.0493, -0.0887,  0.1404,  1.7528],
                      [-0.2755,  0.8267,  0.8030, -0.5056]]),
       ),
)

In [13]:
%timeit b.a = new_value

614 ns ± 9.74 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.05 µs ± 34.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.9 µs ± 128 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

277 µs ± 6.27 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

274 µs ± 10.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f6a8c4de610>
├── 'a' --> tensor([[[ 0.7525, -0.1259,  0.0254],
│                    [-0.0361,  0.7473,  1.2433]],
│           
│                   [[ 0.7525, -0.1259,  0.0254],
│                    [-0.0361,  0.7473,  1.2433]],
│           
│                   [[ 0.7525, -0.1259,  0.0254],
│                    [-0.0361,  0.7473,  1.2433]],
│           
│                   [[ 0.7525, -0.1259,  0.0254],
│                    [-0.0361,  0.7473,  1.2433]],
│           
│                   [[ 0.7525, -0.1259,  0.0254],
│                    [-0.0361,  0.7473,  1.2433]],
│           
│                   [[ 0.7525, -0.1259,  0.0254],
│                    [-0.0361,  0.7473,  1.2433]],
│           
│                   [[ 0.7525, -0.1259,  0.0254],
│                    [-0.0361,  0.7473,  1.2433]],
│           
│                   [[ 0.7525, -0.1259,  0.0254],
│                    [-0.0361,  0.7473,  1.2433]]])
└── 'x' --> <FastTreeValue 0x7f6a8c50ffa0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

45.4 µs ± 638 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f69e2203f70>
├── 'a' --> tensor([[ 0.7525, -0.1259,  0.0254],
│                   [-0.0361,  0.7473,  1.2433],
│                   [ 0.7525, -0.1259,  0.0254],
│                   [-0.0361,  0.7473,  1.2433],
│                   [ 0.7525, -0.1259,  0.0254],
│                   [-0.0361,  0.7473,  1.2433],
│                   [ 0.7525, -0.1259,  0.0254],
│                   [-0.0361,  0.7473,  1.2433],
│                   [ 0.7525, -0.1259,  0.0254],
│                   [-0.0361,  0.7473,  1.2433],
│                   [ 0.7525, -0.1259,  0.0254],
│                   [-0.0361,  0.7473,  1.2433],
│                   [ 0.7525, -0.1259,  0.0254],
│                   [-0.0361,  0.7473,  1.2433],
│                   [ 0.7525, -0.1259,  0.0254],
│                   [-0.0361,  0.7473,  1.2433]])
└── 'x' --> <FastTreeValue 0x7f69e219b580>
    └── 'c' --> tensor([[ 0.7399, -0.1499, -1.7815,  0.3320],
                        [-1.0493, -0.0887,  0.1404,  1.7528],
                 

In [23]:
%timeit t_cat(trees)

40.6 µs ± 879 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

79.9 µs ± 478 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.7399, -0.1499, -1.7815,  0.3320],
                       [-1.0493, -0.0887,  0.1404,  1.7528],
                       [-0.2755,  0.8267,  0.8030, -0.5056]],
              
                      [[ 0.7399, -0.1499, -1.7815,  0.3320],
                       [-1.0493, -0.0887,  0.1404,  1.7528],
                       [-0.2755,  0.8267,  0.8030, -0.5056]],
              
                      [[ 0.7399, -0.1499, -1.7815,  0.3320],
                       [-1.0493, -0.0887,  0.1404,  1.7528],
                       [-0.2755,  0.8267,  0.8030, -0.5056]],
              
                      [[ 0.7399, -0.1499, -1.7815,  0.3320],
                       [-1.0493, -0.0887,  0.1404,  1.7528],
                       [-0.2755,  0.8267,  0.8030, -0.5056]],
              
                      [[ 0.7399, -0.1499, -1.7815,  0.3320],
                       [-1.0493, -0.0887,  0.1404,  1.7528],
                       [-0.2755,  0.8267,  0.8030, -0.5056]],

In [26]:
%timeit Batch.stack(batches)

100 µs ± 2.14 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.7399, -0.1499, -1.7815,  0.3320],
                      [-1.0493, -0.0887,  0.1404,  1.7528],
                      [-0.2755,  0.8267,  0.8030, -0.5056],
                      [ 0.7399, -0.1499, -1.7815,  0.3320],
                      [-1.0493, -0.0887,  0.1404,  1.7528],
                      [-0.2755,  0.8267,  0.8030, -0.5056],
                      [ 0.7399, -0.1499, -1.7815,  0.3320],
                      [-1.0493, -0.0887,  0.1404,  1.7528],
                      [-0.2755,  0.8267,  0.8030, -0.5056],
                      [ 0.7399, -0.1499, -1.7815,  0.3320],
                      [-1.0493, -0.0887,  0.1404,  1.7528],
                      [-0.2755,  0.8267,  0.8030, -0.5056],
                      [ 0.7399, -0.1499, -1.7815,  0.3320],
                      [-1.0493, -0.0887,  0.1404,  1.7528],
                      [-0.2755,  0.8267,  0.8030, -0.5056],
                      [ 0.7399, -0.1499, -1.7815,  0.3320],
                   

In [28]:
%timeit Batch.cat(batches)

183 µs ± 1.3 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

557 µs ± 6.37 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
